# Dimensionality and PCA

In [1]:
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')

# Install torchextractor for facilitating feature extraction
!pip install torchextractor

# Import packages
import torchextractor as tx
import torch
from torchvision import models, transforms
from PIL import Image

import torchextractor as tx
from tqdm import tqdm

from sklearn.decomposition import PCA

Downloading...
From (original): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS
From (redirected): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS&confirm=t&uuid=4c05ecf5-6bcd-40a6-a1f3-02d9f0e5b8b6
To: /content/natural_scenes_demo.tar.gz
100% 395M/395M [00:01<00:00, 222MB/s]  
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown ex

In [5]:
# Download the dataset
!gdown 1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS

# Extract the dataset
!mkdir nsd_data && tar -xzf file.tar.gz -C nsd_data

First, extract layer activations from Alexnet fc7

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pre-trained model  
model = models.alexnet(pretrained=True)
model.to(device).eval()

# Define layers to extract
layers = ["classifier.4"]  # e.g. AlexNet fc7

# Wrap the model with torchextractor
model = tx.Extractor(model, layers)

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize list to hold activations
activations_list = []

# 
import pandas as pd

df = pd.read_csv('natural_scenes_demo/stimulus_data.csv')
image_labels = df['label'].values

# Loop over images in the dataset to extract activations
for img_idx in tqdm(range(100)):  # Example for 100 images

    image_path = f'natural_scenes_demo/image_{img_idx:03d}.jpg'
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        _, activations = model(image)

    activations_list.append(activations['classifier.4'].squeeze().data.cpu().numpy())

